In [1]:
import numpy as np
import holoviews as hv; hv.extension('bokeh', logo=False)
import panel as pn;     pn.extension()
from panel.interact import interact

import sympy as sp
from IPython.display import display, Latex, Math

from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main

def format_matrix_with_parentheses(A):
    A_latex = sp.latex(A)
    return A_latex.replace("\\begin{bmatrix}", "\\begin{pmatrix}").replace("\\end{bmatrix}", "\\end{pmatrix}")

%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [2]:
%%julia
using Pkg, Revise
gla_dir = "../GenLinAlgProblems"
Pkg.activate(gla_dir)
using GenLinAlgProblems, LinearAlgebra, LaTeXStrings, Latexify, Markdown

  Activating project at `~/elementary-linear-algebra/GenLinAlgProblems`



<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">Schur Decomposition: A Gateway to Eigenvalues and Stability</strong>
</div>
    

# 1. Introduction

## 1.1 Schur's Lemma

The eigendecomposition of a square matrix $A$ of size $N \times N$ exists only if $A$ has a complete set<br>
of $N$ linearly independent eigenvectors.

One approach to address this even when the matrix is degenerate<br> (and actually the most useful in numerical analysis) is using a Schur decomposition:<br>
Instead of a **diagonal form**, this decomposition obtains an **upper triangular form** of the matrix $A$.<br>
Interestingly, this can be achieved with a set of orthonormal basis vectors.

<div style="float:left;width:100%;background-color:#F2F5A9;color:black;">

**Schur's Lemma:** Any square matrix $A$ has the form $\;\; A = Q T Q^H,\;\;$
where
- $Q$ is unitary, i.e.,  $Q^H Q = I$
- $T$ is upper triangular
</div>

**Examples:**
* $A=\begin{pmatrix} 4 & \;\;1 \\ 2 & \;\;3\end{pmatrix}, \quad
Q = \frac{1}{\sqrt{2}} \left(\begin{array}{rr} 1 & -1 \\ 1 & 1 \end{array}\right), \quad T = \begin{pmatrix}5 & 1 \\ 0 & 2\end{pmatrix}\;\;$ satisfies $A = Q\ T\ Q^t$
* $A=\left(\begin{array}{rr} 1 & -3 \\ 2 & 4\end{array}\right), \quad
Q = \frac{1}{\sqrt{2}} \left(\begin{array}{rr}  i & -i \\ 1 & 1  \end{array}\right), \quad T = \begin{pmatrix} 1 + 3i & 0 \\ 0 & 1 - 3i\end{pmatrix}\;\;$ satisfies $A = Q\ T\ Q^t$

## 1.2 A Constructive Proof

The key idea of Schur's Lemma is to **iteratively construct the triangular matrix** $T$<br> using
orthogonal matrices $𝑄_i$ while maintaining the similarity transform $A = Q\ T\ Q^t$

### 1.2.1 Use an Eigenvector to Introduce Zeros in the First Column

**Reminder:** Any $N \times N$ matrix $A$ has at least one eigenpair $(\lambda,x)$ for every distinct eigenvalue $\lambda$.<br><br>
Let us chose one such eigenpair, and without loss of generality assume that $x$ has unit length, i.e., $x^t x = 1$.<br><br>
Extend $\left\{ x \right\}$ to a full orthonormal basis of $N$ vectors. Note that since $A$ may have complex eigenvalues,<br>
$\qquad$ the resulting matrix
$Q = \begin{pmatrix} x &q_2&q_3&\dots &q_N\end{pmatrix} = \begin{pmatrix} x & \tilde{Q}\end{pmatrix}$
may have complex entries.

$\qquad \begin{aligned}
Q^H A Q &= \begin{pmatrix} x^H \\ \tilde{Q}^H \end{pmatrix}\  A\ \begin{pmatrix} x & &\tilde{Q}\end{pmatrix}\qquad & \\
&=         \begin{pmatrix} x^H \\ \tilde{Q}^H \end{pmatrix} \begin{pmatrix} A x & A \tilde{Q} \end{pmatrix} & \text{ now use } A x = \lambda x \;\; \text{ and } x^t x = 1 & \\
&=         \begin{pmatrix} \lambda & x^H A \tilde{Q}^H \\
                           \lambda \tilde{Q}^H x & \tilde{Q}^H A \tilde{Q}\end{pmatrix} &
                           \text{ but } x \perp q_2, q_3, \dots q_n \\ 
&=          \begin{pmatrix} \lambda & x^H A \tilde{Q}^H \\
                           0 & \tilde{Q}^H A \tilde{Q}\end{pmatrix} &
\end{aligned}$

Observe that $\tilde{a}^H = x^H A \tilde{Q}^H$ is a row vector, and $\;\;\tilde{A} = \tilde{Q}^H A \tilde{Q}\;\;$ is a matrix of size $(N-1) \times (N-1)$, i.e.,

$\qquad Q^H A Q = \begin{pmatrix} \lambda & \tilde{a}^H \\ 0 & \tilde{A} \end{pmatrix}$.

### 1.2.2 Repeat with $\tilde{A}$

To see that we can continue this process with successively smaller matrices $\tilde{A}$:
* Let $Q_1^H A Q_1 = \begin{pmatrix} \lambda_1 & \tilde{a_1}^H \\ 0 & \tilde{A}_1 \end{pmatrix}$
* obtain $\;\;\tilde{Q}_2^H \tilde{A}_1 \tilde{Q}_2 = \begin{pmatrix} \lambda_2 & \tilde{a}_2^H \\ 0 & \tilde{A}_2 \end{pmatrix}$ for some given eigenpair $(\lambda_2, x_2)$ of $\tilde{A}_1$
* Set $Q_2 = \begin{pmatrix} 1 & 0 \\ 0 & \tilde{Q_2}\end{pmatrix}\;\;$ and therefore
$\;\;(Q_2 Q_1)^H A (Q_1 Q_2) = \begin{pmatrix} \lambda_1 & \dots & \dots \\
                0          & \lambda_2 & \dots \\
                0          & 0 & \tilde{A}_3 \end{pmatrix}$
* repeat this process for each matrix $\tilde{A}_i,\;\;$ resulting in an upper triangular matrix<br>
  $\;\;T = (Q_N \dots Q_2 Q_1)^H A (Q_1 Q_2) =
  \begin{pmatrix} \lambda_1 & \dots     & \dots  &  \dots  &  \dots \\
                 0          & \lambda_2 & \dots  &  \dots  &  \dots \\
                 0          & 0         & \ddots &  \dots  &  \dots \\
                 0          & 0         &  0     & \ddots  &  \dots \\
                 0          & 0         &  0     & \dots & \lambda_N
  \end{pmatrix}$

In [ ]:
%%julia
function works_householder_matrix(v::AbstractVector{T}, i::Integer) where T
    if T <: Integer  v = convert.(Rational{T}, v) end

    n = length(v)
    1 <= i <= n || throw(ArgumentError("Pivot index must be between 1 and $(n)"))

    x                        = v[i:end]
    below_pivot_squared_norm = sum(x[2:end] .^ 2)

    if iszero(below_pivot_squared_norm) return Matrix{eltype(v)}(I, n, n) end

    s              = x[1] >= 0 ? one(eltype(x)) : -one(eltype(x))
    x_squared_norm = x[1]^2 + below_pivot_squared_norm
    e1             = [one(eltype(x)); zeros(eltype(x), length(x) - 1)]
    alpha          = s * sqrt(x_squared_norm)
    u              = x - alpha * e1

    u_squared_norm = sum(u .^ 2)

    if iszero(u_squared_norm)
        return Matrix{eltype(v)}(I, n, n)
    end

    H = I - (2 / u_squared_norm) * (u * u')

    full_H = Matrix{eltype(H)}(I, n, n)
    full_H[i:end, i:end] = H
    return full_H
end

In [51]:
%%julia
function householder_matrix(v::AbstractVector{T}, i::Integer) where T
    if T <: Integer
        v = convert.(Rational{T}, v)
    end

    n = length(v)
    1 <= i <= n || throw(ArgumentError("Pivot index must be between 1 and $(n)"))

    x = v[i:end]
    below_pivot_squared_norm = sum(x[2:end] .^ 2)

    if iszero(below_pivot_squared_norm) && x[1] >= zero(eltype(x))
        return Matrix{eltype(v)}(I, n, n)
    end

    x_squared_norm = x[1]^2 + below_pivot_squared_norm
    u = copy(x)
    u[1] += sign(x[1]) * sqrt(x_squared_norm)

    u_squared_norm = 2 * (u[1]^2)

    if iszero(u_squared_norm)
        return Matrix{eltype(v)}(I, n, n)
    end

    H = I - (2 / u_squared_norm) * (u * u')

    full_H = Matrix{eltype(H)}(I, n, n)
    full_H[i:end, i:end] = H
    return full_H
end
;

In [55]:
%%julia
A  = [4 1 2; 1 3 5; 2 5 6]*1.
AA = copy(A)
Q1 = works_householder_matrix( A[:,1], 1 )
py_show( round.(Q1,digits=2), round.(Q1*A,digits=2), round.(Q1*A*Q1',digits=2) )
#Q, T = naive_schur_decomposition(A)
#py_show( L"A = ", round.(Q,digits=3), round.(T,digits=3), round.(Q',digits=3) )

<IPython.core.display.Latex object>

In [36]:
%%julia
A  = [4 1 2; 1 3 5; 2 5 6]
Q1 = householder_matrix( A[:,1], 1 )
py_show( Q1, Q1*A )

x = Rational{Int64}[4, 1, 2]
u = [-0.5825756949558398, 1.0, 2.0]


<IPython.core.display.Latex object>

In [15]:
%%julia
"""
Compute the Schur decomposition of a matrix A using iterative
Householder transformations.

Returns:
    Q: Unitary matrix
    T: Upper triangular matrix
"""
function naive_schur_decomposition(A::Matrix{Float64})
    m, n = size(A)
    Q = I(m)                          # Initialize Q as the identity matrix
    T = copy(A)                       # Start with A

    for j in 1:n-1
        # Extract the subvector from column j
        v = T[:, j]
        v[1:j-1] .= 0.0  # Zero out elements above the diagonal

        # Check if transformation is needed
        if norm(v[j:end]) > 1e-10
            # Compute the Householder matrix
            H = compute_householder(v, j)

            # Update T and Q
            T = H * T * H'
            Q = Q * H'
        end
    end

    return Q, T
end
;

In [ ]:
# Example: Visualize matrices Q and T
A = np.array([[4, 1], [2, 3]], dtype=float)
Main.A = A  # Pass matrix to Julia
Q, T = Main.schur_decomposition(Main.A)

# Visualize
q_plot = matrix_visualization(Q, title="Matrix Q")
t_plot = matrix_visualization(T, title="Matrix T")
pn.Row(q_plot, t_plot).servable()
    

In [ ]:
# Interactive Schur decomposition exploration
def compute_schur(matrix):
    Main.A = matrix
    Q, T = Main.schur_decomposition(Main.A)
    return Q, T

@pn.depends()
def interactive_plot(matrix):
    Q, T = compute_schur(matrix)
    q_plot = matrix_visualization(Q, title="Matrix Q")
    t_plot = matrix_visualization(T, title="Matrix T")
    return pn.Row(q_plot, t_plot)

matrix_input = pn.widgets.TextInput(name="Matrix (comma-separated rows)", value="4,1;2,3")
pn.Column(matrix_input, interactive_plot).servable()
    

# 3. Examples

### Example 1: Simple Matrix
Let $A = \begin{bmatrix} 4 & 1 \\ 2 & 3 \end{bmatrix}$.
The Schur decomposition yields:

$Q = \begin{bmatrix} ... \end{bmatrix}$

$T = \begin{bmatrix} ... \end{bmatrix}$

Try it interactively above.
    

# 4. Take Away

### Summary
- The Schur decomposition expresses a square matrix $A$ as $A = Q T Q^H$.
- The diagonal of $T$ contains the eigenvalues of $A$.
- $Q$ is unitary, preserving numerical stability.

### Why Use Schur Decomposition?

While eigendecomposition also expresses a matrix in terms of its eigenvalues and eigenvectors, it has some limitations:
1. **Existence**:
   - Eigendecomposition exists only for diagonalizable matrices. Non-diagonalizable matrices, such as those with defective eigenvalues, cannot be decomposed using eigendecomposition.
   - Schur decomposition, however, exists for every square matrix, regardless of diagonalizability.

2. **Stability**:
   - Schur decomposition involves unitary matrices, which are numerically stable because they preserve lengths and angles during computations. This is especially important in applications requiring high precision.

3. **Practicality**:
   - The Schur decomposition retains more structure in the triangular matrix \( T \) compared to eigendecomposition, where the matrix is diagonal. This makes it a versatile tool in numerical linear algebra, particularly in iterative algorithms like QR or spectral analysis.

### Applications of Schur Decomposition

The Schur decomposition is fundamental in various areas, including:
- **Eigenvalue Computation**: Eigenvalues of $A$ are readily found as the diagonal elements of $T$.
- **Control Theory**: Stability of a system can be analyzed by examining the eigenvalues of $A$ through its Schur form.
- **Spectral Analysis**: Provides a basis for understanding the spectrum of a matrix.
- **Matrix Functions**: Facilitates the computation of matrix exponentials and logarithms.

In [ ]:
%%julia
using LinearAlgebra
using Random

function schur_decomposition(A::Matrix{Float64})
    Q, T = schur(A, :Schur)
    return Q, T
end
    

In [64]:
%%julia
A=[1 -3; 3 1]
E=eigen(A)
py_show( E.values[1], E.vectors[:,1])

RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: Unsupported type: Vector{ComplexF64}
Stacktrace:
  [1] error(s::String)
    @ Base ./error.jl:35
  [2] (::GenLinAlgProblems.var"#f#13"{Nothing})(x::Vector{ComplexF64})
    @ GenLinAlgProblems ~/elementary-linear-algebra/GenLinAlgProblems/src/LatexRepresentations.jl:113
  [3] (::GenLinAlgProblems.var"#11#17"{Symbol, GenLinAlgProblems.var"#f#13"{Nothing}})(arg::Vector{ComplexF64})
    @ GenLinAlgProblems ~/elementary-linear-algebra/GenLinAlgProblems/src/LatexRepresentations.jl:132
  [4] map(f::GenLinAlgProblems.var"#11#17"{Symbol, GenLinAlgProblems.var"#f#13"{Nothing}}, t::Tuple{ComplexF64, Vector{ComplexF64}})
    @ Base ./tuple.jl:356
  [5] L_show(::ComplexF64, ::Vararg{Any}; arraystyle::Symbol, color::Nothing, number_formatter::Nothing, inline::Bool)
    @ GenLinAlgProblems ~/elementary-linear-algebra/GenLinAlgProblems/src/LatexRepresentations.jl:132
  [6] L_show(::ComplexF64, ::Vararg{Any})
    @ GenLinAlgPr

In [63]:
%%julia
E.values

array([1.-3.j, 1.+3.j])